In [1]:
import pandas as pd
import os
import numpy as np
from sklearn.preprocessing import LabelEncoder
path = "C:\Users\DEEPSAGAR\Airbnb\\"
file = ['train_users.csv', 'age_gender_bkts.csv', 'sessions.csv', 'countries.csv', 'test_users.csv']
data = {}
### Get data ###
for f in file:
    data[f.replace('.csv','')]=pd.read_csv(path+f)
    #print(f)
    #print(data[f.replace('.csv','')].columns)

In [2]:
### Variables ###
train = data['train_users']
test = data['test_users']
age = data['age_gender_bkts']

sessions = data['sessions']
sessions['id'] = sessions['user_id']
sessions = sessions.drop(['user_id'],axis=1)
country = data['countries']
target = train['country_destination']
train = train.drop(['country_destination'],axis=1)

In [3]:
country

,country_destination,lat_destination,lng_destination,distance_km,destination_km2,destination_language,language_levenshtein_distance
0,AU,-26.853388,133.275160,15297.7440,7741220,eng,0.00
1,CA,62.393303,-96.818146,2828.1333,9984670,eng,0.00
2,DE,51.165707,10.452764,7879.5680,357022,deu,72.61
3,ES,39.896027,-2.487694,7730.7240,505370,spa,92.25
4,FR,46.232193,2.209667,7682.9450,643801,fra,92.06
5,GB,54.633220,-3.432277,6883.6590,243610,eng,0.00
6,IT,41.873990,12.564167,8636.6310,301340,ita,89.40
7,NL,52.133057,5.295250,7524.3203,41543,nld,63.22
8,PT,39.553444,-7.839319,7355.2534,92090,por,95.45
9,US,36.966427,-95.844030,0.0000,9826675,eng,0.00


In [4]:
del data
##### Merging #####
print(len(train))
train1 = pd.merge(train,age,on='gender')
test1 = pd.merge(test,age,on='gender')
train2 = pd.merge(train,sessions,on='id')
test2 = pd.merge(test,sessions,on='id')
print(len(train1))
id = train['id']
train = train.drop(['id'],axis=1)
train = train.fillna(0)
test = test.fillna(0)
id_test = test['id']
print('size of id:\t',len(id_test))
test = test.drop(['id'],axis=1)
labelencoder={}

213451
0
('size of id:\t', 62096)


In [5]:
def labelencoderCustom(data,labelencoder={}):
    result = []
    for i in data:
        if i not in labelencoder:
            labelencoder[i] = len(labelencoder)+1
            result.append(labelencoder[i])
        else:
            result.append(labelencoder[i])
    return result,labelencoder
print('Type of columns')
for col in train.columns:
    if train[col].dtypes == object :
        #train = train.drop([col],axis=1)
        #le = LabelEncoder()
        #print(set(train[col]))
        #le.fit(list(train[col].values)+list(test[col].values))
        
        train[col],labelencoder = labelencoderCustom(train[col].values,labelencoder)
        test[col],labelencoder =  labelencoderCustom(test[col].values,labelencoder)
print('Columns:')
print(train.dtypes)
print('Columns test:')
print(test.dtypes)
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=600)
rf = rf.fit(train,target)


Type of columns
Columns:
date_account_created         int64
timestamp_first_active       int64
date_first_booking           int64
gender                       int64
age                        float64
signup_method                int64
signup_flow                  int64
language                     int64
affiliate_channel            int64
affiliate_provider           int64
first_affiliate_tracked      int64
signup_app                   int64
first_device_type            int64
first_browser                int64
dtype: object
Columns test:
date_account_created         int64
timestamp_first_active       int64
date_first_booking           int64
gender                       int64
age                        float64
signup_method                int64
signup_flow                  int64
language                     int64
affiliate_channel            int64
affiliate_provider           int64
first_affiliate_tracked      int64
signup_app                   int64
first_device_type            int64
fi

In [6]:
sample_submission = {}
sample_submission['id'] = id_test
#print(len(test))
#print(len(sample_submission['id'].values))
#print(sample_submission.columns)
sample_submission['country'] = rf.predict(test)
s = pd.DataFrame.from_dict(sample_submission)
s.to_csv('sub.csv',index=False)